In [ ]:
%env LLM_BASE_URL=https://dashscope.aliyuncs.com/compatible-mode/v1
%env LLM_API_KEY=替换为自己的Qwen API Key，如果不需要评估，则不需要

In [2]:
import os
import pandas as pd
from openai import OpenAI

# 加载数据

In [3]:
expr_version = 'product_v02_flowise_hyde'

preprocess_output_dir = os.path.join(os.path.pardir, 'outputs', 'v1_20240713')
expr_dir = os.path.join(os.path.pardir, 'experiments', expr_version)

In [4]:
qa_df = pd.read_excel(os.path.join(preprocess_output_dir, 'question_answer.xlsx'))
prediction_df = qa_df[qa_df['dataset'] == 'test'][['uuid', 'question', 'qa_type', 'answer']].rename(columns={'answer': 'ref_answer'})

# 调用Flowise API

In [7]:
import requests

API_URL = "http://192.168.31.92:3000/api/v1/prediction/31731cbf-2dd2-45e4-8b26-a065b6739dcd"
def rag(question):
    payload = {
        "question": question
    }
    response = requests.post(API_URL, json=payload)
    return response.json()

In [8]:
rag('这份报告的发布时间是什么时候')

{'text': '这份报告的发布时间是2023年12月12日。',
 'question': '这份报告的发布时间是什么时候',
 'chatId': '956b7e56-cbe1-4df3-adf8-7c5f70a53432',
 'chatMessageId': '0a3b34c9-8246-40ba-9362-243df0fa0272',
 'isStreamValid': False,
 'sessionId': '956b7e56-cbe1-4df3-adf8-7c5f70a53432'}

# 批量预测

In [9]:
from tqdm.auto import tqdm

answer_dict = {}
for idx, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
    uuid = row['uuid']
    question = row['question']
    raw_resp = rag(question)

    answer_dict[question] = {
        'uuid': uuid,
        'ref_answer': row['ref_answer'],
        'gen_answer': raw_resp['text']
    }

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
prediction_df.loc[:, 'gen_answer'] = prediction_df['question'].apply(lambda q: answer_dict[q]['gen_answer'])

In [11]:
prediction_df.sample(5)

,uuid,question,qa_type,ref_answer,gen_answer
180,a52f60d5-4430-405f-a2dd-8a975ac03b91,美国金融危机风险指标的英文缩写是什么？,detailed,ROFCI,美国金融危机风险指标的英文缩写是ROFCI。
10,41d95288-441d-4c02-948a-6a3f0f4ef3ba,2023年全球供应链恢复情况如何？,detailed,持续恢复,2024年全球供应链和工业生产预计将在波动中复苏，但支撑2023年经济增长的需求端因素可能在...
365,2accb7dd-1bd7-42c9-89ad-a32b6725c95a,2024年全球资金状况和货币市场基金的前景如何？,large_context,2024年，主要经济体将结束加息周期，资金流动性会呈现“紧平衡”状态，可能导致货币市场脆弱性...,"2024年全球资金状况可能呈现紧平衡状态，资金""紧平衡""可能导致货币市场短期资金成本大幅抬升..."
173,05696bdf-74d8-4788-8e18-86439fa0bf0c,欧洲央行在3月16日的利率是多少百分比?,detailed,3.5%,欧洲央行在3月16日的变动后利率是3.5%。
19,1f406690-b478-43cd-96f8-cd77924e300e,2023年三季度英国GDP环比增速是多少？,detailed,2023年三季度英国GDP环比增速下降0.03%。,2023年三季度英国GDP环比增速为0.03%。


# 评估

In [12]:
from langchain_openai import ChatOpenAI

judge_llm = ChatOpenAI(
    api_key=os.environ['LLM_API_KEY'],
    base_url=os.environ['LLM_BASE_URL'],
    model_name='qwen2-72b-instruct',
    temperature=0
)

import time

def evaluate(prediction_df):
    """
    对预测结果进行打分
    :param prediction_df: 预测结果，需要包含问题，参考答案，生成的答案，列名分别为question, ref_answer, gen_answer
    :return 打分模型原始返回结果
    """
    prompt_tmpl = """
你是一个经济学博士，现在我有一系列问题，有一个助手已经对这些问题进行了回答，你需要参照参考答案，评价这个助手的回答是否正确，仅回复“是”或“否”即可，不要带其他描述性内容或无关信息。
问题：
<question>
{{question}}
</question>

参考答案：
<ref_answer>
{{ref_answer}}
</ref_answer>

助手回答：
<gen_answer>
{{gen_answer}}
</gen_answer>
请评价：
    """
    results = []

    for _, row in tqdm(prediction_df.iterrows(), total=len(prediction_df)):
        question = row['question']
        ref_answer = row['ref_answer']
        gen_answer = row['gen_answer']

        prompt = prompt_tmpl.replace('{{question}}', question).replace('{{ref_answer}}', str(ref_answer)).replace('{{gen_answer}}', gen_answer).strip()
        result = judge_llm.invoke(prompt).content
        results.append(result)

        time.sleep(1)
    return results

In [13]:
prediction_df['raw_score'] = evaluate(prediction_df)

  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
prediction_df['raw_score'].unique()

array(['是', '否'], dtype=object)

In [15]:
prediction_df['score'] = (prediction_df['raw_score'] == '是').astype(int)

In [16]:
prediction_df['score'].mean()

0.75